In [ ]:
import helper
import time
import numpy
import cv2

In [ ]:
def run_test(image_link, victim_position, fatals, victim_needs, rescue_position, rescue_resources):
    start_time = time.time()
    WIDTH, HEIGHT = 640, 480
    background = numpy.array(Image.open(image_link))
    background = cv2.resize(background, (WIDTH, HEIGHT))
    background = numpy.transpose(background, (1, 0, 2))
    groundMapModel = helper.buildMap(background)
    # print(groundMapModel)
    groundMapModel = numpy.repeat(groundMapModel.reshape(groundMapModel.shape[0], groundMapModel.shape[1], 1), 3, axis=2)
    groundMapModel = helper.turn_image_to_binary(groundMapModel, 1)

    res = numpy.array(rescue_position)
    vic = numpy.array(victim_position)

    paths = helper.solve_for_paths(
        groundMapModel, res, vic, fatals, rescue_resources, victim_needs
    )

    duration = time.time() - start_time
    return paths, duration